In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import pickle

In [ ]:
MS_PER_S = 1000
PATH_TRAIN_LOGS = "./data/external/train_logs.csv"

In [ ]:
def extract(path):

    X = pd.read_csv(path)
    X = X.sort_values(["id", "event_id"], ascending=[True, True])
    
    return X

In [ ]:
def scrub_activity(X):

    # 'Move From' activity recorded with low-level cursor loc details
    # extract bigger-picture 'Move From'
    # QUESTION: what's the difference between Move From, and a cut+paste?
    X['activity_detailed'] = X['activity']
    X.loc[X['activity'].str.contains('Move From'), 'activity'] = 'Move'

    return X

In [ ]:
PAUSE_THRESHOLD_MS = 1000
N_ACTIVITIES_UNTIL_START_WINDOW_CLOSES = 100

def enrich_pauses(X):
    """Must infer pauses, as no explicit record indicates."""

    X['up_time_lag1'] = (
        X
        .groupby(['id'])
        ['up_time']
        .shift(1)
        )
    # latency does not mean a meaningful pause
    X['latency_time'] = (
        X['down_time'] - X['up_time_lag1']
        )

    X['preceding_pause_time'] = X['latency_time']
    # first record lacks preceding_pause_time: that's time before first key press
    X.loc[X['event_id'] == 1, 'preceding_pause_time'] = X['down_time']
    # expect some negative pause times -- interpret as, no real pause
    has_no_real_pause = X['preceding_pause_time'] <= PAUSE_THRESHOLD_MS
    X.loc[has_no_real_pause, 'preceding_pause_time'] = None
    # not obvious how to tag "initial planning pause" 
    # tried "first 5 minutes", but when that pause is 10 minutes, that fails.
    # first XX minutes is fragile
    # first XX events may help -- what's your extent of pause before *action*?
    X['preceding_pause_time_start_window'] = X['preceding_pause_time']
    X.loc[X['event_id'] <= N_ACTIVITIES_UNTIL_START_WINDOW_CLOSES, 'preceding_pause_time_start_window'] = None

    X['total_pause_time'] = (
        X
        .groupby(['id'])
        ['preceding_pause_time']
        .transform('sum')
        )
    X['rolling_pause_time'] = (
        X
        .groupby(['id'])
        ['preceding_pause_time']
        .cumsum()
        )
    X['rolling_pause_time_fraction'] = (
        X['rolling_pause_time'] / X['total_pause_time']
        )

    # summarize pause distr
    # MS_IN_PAUSE_BUCKET_MAX = 200e3
    # PAUSE_BUCKET_STEP_MS = 500
    # X['preceding_pause_time_bucket'] = pd.cut(
    #     X['preceding_pause_time'],
    #     bins=np.arange(
    #         0, 
    #         MS_IN_PAUSE_BUCKET_MAX,
    #         PAUSE_BUCKET_STEP_MS
    #         )
    #     )
    # X['preceding_pause_time_bucket'].value_counts()
    # WARNING: this representation of pause distribution is dense & large
    # a few parameters from distribution model far more succinct

    return X

In [ ]:
# if pause exceeds threshold duration, a "burst" has ended
SECONDS_PER_BURST = 2

def enrich_time_bursts(X):

    X['is_new_burst_start'] = (
        X['preceding_pause_time'] > MS_PER_S * SECONDS_PER_BURST
        ).astype(int)
    X.loc[X['event_id'] == 1, 'is_new_burst_start'] = 1
    X['burst_id'] = (
        X
        .groupby(['id'])
        ['is_new_burst_start']
        .cumsum()
        )
    X['burst_time_start'] = (
        X
        .groupby(['id', 'burst_id'])
        ['down_time']
        .transform('min')
        )
    X['burst_time_end'] = (
        X
        .groupby(['id', 'burst_id'])
        ['up_time']
        .transform('max')
        )
    X['burst_time_duration'] = (
        X['burst_time_end'] - X['burst_time_start']
        )
    
    return X

In [ ]:
def enrich_activity_streaks(X):
        
    # consecutive activity (independent of time) suggests productive writing flow

    X['activity_lag1'] = (
        X
        .groupby(['id'])
        ['activity']
        .shift(1)
        )

    X['is_new_activity_streak_start'] = (
        X['activity'] != X['activity_lag1']
    ).astype(int)
    X.loc[X['event_id'] == 1, 'is_new_activity_streak_start'] = 1

    X['is_activity_streak_end'] = (
        X
        .groupby(['id'])
        ['is_new_activity_streak_start']
        .shift(-1)
        )
    X['is_activity_streak_end'] = X['is_activity_streak_end'].fillna(1) 

    X['activity_streak_id'] = (
        X
        .groupby(['id'])
        ['is_new_activity_streak_start']
        .cumsum()
    )

    X['activity_streak_length_thin'] = (
        X
        .groupby(['id', 'activity_streak_id'])
        .transform('size')
    )
    X.loc[X['is_activity_streak_end'] == 0, 'activity_streak_length_thin'] = None

    return X

In [ ]:
def enrich_word_count(X):

    # word count offers a productivity measure
    X['word_count_lag1'] = (
        X
        .groupby(['id'])
        ['word_count']
        .shift(1)
        )

    X['word_count_delta_event'] = (
        X['word_count'] - X['word_count_lag1']
        )

    X['word_count_delta_burst'] = (
        X
        .groupby(['id', 'burst_id'])
        ['word_count_delta_event']
        .transform('sum')
        )
    # de-duplicate to one value per burst -- easier for downstream aggregation
    X['word_count_delta_burst_thin'] = X['word_count_delta_burst']
    X.loc[X['is_new_burst_start'] == 0, 'word_count_delta_burst_thin'] = None

    return X

In [ ]:
def enrich_cursor_position(X):

    # one-way cursor movement might be most productive
    # jumping around is choppy
    X['cursor_position_lag1'] = (
        X
        .groupby(['id'])
        ['cursor_position']
        .shift(1)
        )

    X['has_cursor_position_moved_right'] = (
        X['cursor_position'] > X['cursor_position_lag1']
        ).astype(int)

    # if cursor position increases due to copy+paste (perhaps of essay prompt),
    # that doesn't reflect grade-driving output
    X['cursor_position_input'] = np.where(
        X['activity'] == "Input", 
        X["cursor_position"], 
        np.nan
        )
    X['cursor_position_cummax'] = (
        X
        .groupby(['id'])
        ['cursor_position_input']
        .cummax()
        )
    # for some reason, unable to chain below statements with above
    X['cursor_position_cummax'] = (
        X
        .groupby(['id'])
        ['cursor_position_cummax']
        .ffill()
        .fillna(0)
    )

    X['cursor_position_vs_max'] = (
        X['cursor_position'] - X['cursor_position_cummax']
        )

    X['cursor_position_last_space'] = np.where(
        (X['activity'] == "Input") & (X["text_change"] == ' '),
        X['cursor_position'],
        np.nan
    ) 
    X['cursor_position_last_space'] = (
        X
        .groupby(['id'])
        ['cursor_position_last_space']
        .ffill()
        # likely not beginning essay with a space
        .fillna(0)
    )

    X = X.drop(columns='cursor_position_input')

    return X

In [ ]:
def enrich_word_length(X):
        
    # word length offers a content quality measure.
    # hard to track entire words sequence in rolling fashion.
        # every word's length, in a list of one element per word?  
    # more tractable to track very latest string

    is_edit_to_latest_string = (
        X['cursor_position'] > X['cursor_position_last_space']
    )

    X['is_latest_space'] = (
        (X['cursor_position_vs_max'] == 0)
        & (X['activity'] == "Input")
        & (X["text_change"] == ' ')
        )

    X['is_latest_string_end'] = (
        X
        .groupby(['id'])
        ['is_latest_space']
        .shift(-1)
        # last process records
        .fillna(True)
        )

    X['n_alphanum_char_added_to_latest_string'] = 0
    is_alphanumeric_addition = (
        (X['activity'] == "Input")
        & (X["text_change"] == 'q')
        )
    X.loc[
        (is_alphanumeric_addition & is_edit_to_latest_string), 
        'n_alphanum_char_added_to_latest_string'
        ] = 1
    is_alphanumeric_subtraction = (
        (X['activity'] == "Remove/Cut")
        & (X['up_event'] == 'Backspace')
        & (X["text_change"] == 'q')
        )
    X.loc[
        (is_alphanumeric_subtraction & is_edit_to_latest_string), 
        'n_alphanum_char_added_to_latest_string'
        ] = -1

    # example: 2nd string, 2 characters in.
    # considering cumsum for each character in 2nd string, 
    # subtract those characters from 1st
    X['rolling_length_strings'] = (
        X
        .groupby(['id'])
        ['n_alphanum_char_added_to_latest_string']
        .cumsum() 
        ) 

    X['rolling_length_completed_strings'] = None
    X.loc[
        X['is_latest_space'], 'rolling_length_completed_strings'
        ] = X['rolling_length_strings']
    X['rolling_length_completed_strings'] = (
        X
        .groupby(['id'])
        ['rolling_length_completed_strings']
        .ffill()
        .fillna(0)
    )

    X['rolling_length_latest_string'] = (
        X['rolling_length_strings'] 
        - X['rolling_length_completed_strings']
    )

    X['length_latest_string'] = None
    X.loc[
        X['is_latest_string_end'], 'length_latest_string'
        ] = X['rolling_length_latest_string']
    
    return X

In [ ]:
def enrich_punctuation(X):
        
    # if thoughts aren't separated by punctuation, writing won't score well
    X['is_thought_delimiting_punctuation'] = (
        (X['text_change'] == ".")
        | (X['text_change'] == ". ")
        | (X['text_change'] == ",")
        | (X['text_change'] == "-")
        | (X['text_change'] == "!")
        | (X['text_change'] == ";")
        | (X['text_change'] == "?")
        | (X['text_change'] == ":")
        ).astype(int)

    X['is_special_punctuation'] = (
        (X['text_change'] == "=")
        | (X['text_change'] == "/")
        | (X['text_change'] == "\\")
        | (X['text_change'] == "(")
        | (X['text_change'] == ")")
        | (X['text_change'] == "\n")
        | (X['text_change'] == "[")
        | (X['text_change'] == "]")
        | (X['text_change'] == ">")
        | (X['text_change'] == "<")
        | (X['text_change'] == "$")
        | (X['text_change'] == "*")
        | (X['text_change'] == "&")
    )

    return X

In [ ]:
TOTAL_MIN_MAX_EXPECTED = 30
TOTAL_MIN_PLUS_BUFFER = 150 # id 21bbc3f6 case extended to 140 min ... odd
SECONDS_PER_MIN = 60
SECONDS_PER_WINDOW = 30

def enrich_time_windows(X):

    # windows allow for time-sequence features
    # expect that some essays extend beyond 30 min described in 'Data Collection'
    # downstream, **do not tabulate over a writer's unused time windows**!!

    X['window_30s'] = pd.cut(
        X['down_time'],
        bins=np.arange(
            0, 
            TOTAL_MIN_PLUS_BUFFER * SECONDS_PER_MIN * MS_PER_S, 
            SECONDS_PER_WINDOW * MS_PER_S
            )
        )

    X['is_time_beyond_expected_max'] = (
        X['up_time'] > TOTAL_MIN_MAX_EXPECTED * SECONDS_PER_MIN * MS_PER_S
    ).astype(int)

    return X

In [ ]:
ACTIVITY_CATEGORIES = ['Nonproduction', 'Input', 'Remove/Cut', 'Replace', 'Paste', 'Move']

def transform_activity_onehot(X, is_training_run):

    if is_training_run:

        pipeline = ColumnTransformer(
            transformers=[(
                'onehot_encode', 
                preprocessing.OneHotEncoder(
                    categories=[ACTIVITY_CATEGORIES], 
                    sparse=False, 
                    handle_unknown='infrequent_if_exist'
                    ),
                ["activity"]
            )],
            remainder='passthrough',
            verbose_feature_names_out=False
            )
        
        pipeline.fit(X)

        with open("pipeline_activity_onehot.pkl", "wb") as f:
            pickle.dump(pipeline, f)

    else:
        with open("pipeline_activity_onehot.pkl", "rb") as f:
            pipeline = pickle.load(f)

    original_categorical = X['activity']

    X_dtypes = X.dtypes.to_dict()
    X = pipeline.transform(X)
    X = pd.DataFrame(X, columns=pipeline.get_feature_names_out())
    X = pd.concat([X, original_categorical], axis=1)
    X = X.astype(X_dtypes)

    return X

In [ ]:
def enrich_burst_type(X):

    for activity in ACTIVITY_CATEGORIES:

        X['burst_events_' + activity] = (
            X
            .groupby(['id', 'burst_id'])
            ['activity_' + activity]
            .transform('sum')
            ).astype(float)
        
    X['burst_type'] = (
        X
        [['burst_events_' + activity for activity in ACTIVITY_CATEGORIES]]
        .idxmax(axis=1)
        )
    X['burst_type'] = (
        X['burst_type']
        .str
        .replace("burst_events_", "", regex=True)
        )
    
    return X

In [ ]:
def transform_burst_type_onehot(X, is_training_run):

    if is_training_run:
        
        pipeline = ColumnTransformer(
            transformers=[(
                'onehot_encode', 
                preprocessing.OneHotEncoder(
                    categories=[ACTIVITY_CATEGORIES], 
                    sparse=False, 
                    handle_unknown='infrequent_if_exist'
                    ),
                ["burst_type"]
            )],
            remainder='passthrough',
            verbose_feature_names_out=False
            )
        
        pipeline.fit(X)
        
        with open("pipeline_burst_type_onehot.pkl", "wb") as f:
            pickle.dump(pipeline, f)

    else:
        with open("pipeline_burst_type_onehot.pkl", "rb") as f:
            pipeline = pickle.load(f)

    original_categorical = X['burst_type']
    X_dtypes = X.dtypes.to_dict()
    X = pipeline.transform(X)
    X = pd.DataFrame(X, columns=pipeline.get_feature_names_out())
    X = pd.concat([X, original_categorical], axis=1)
    X = X.astype(X_dtypes)

    for activity in ACTIVITY_CATEGORIES:

        X['is_new_burst_start_' + activity] = (
            X['is_new_burst_start'] * 
            X['burst_type_' + activity]
            )
        
        X['is_new_activity_streak_start_' + activity] = (
            X["activity_" + activity] * X['is_new_activity_streak_start']
        )

    return X

In [ ]:
def subset_features(X):

    return X[[
        "id",
        "event_id",
        "is_time_beyond_expected_max",
        "window_30s",
        "burst_id",
        "burst_type",
        "burst_type_Nonproduction",
        "burst_type_Input",
        "burst_type_Remove/Cut",
        "burst_type_Replace",
        "burst_type_Paste",
        "burst_type_Move",
        "is_new_burst_start",
        "is_new_burst_start_Nonproduction",
        "is_new_burst_start_Input",
        "is_new_burst_start_Remove/Cut",
        "is_new_burst_start_Replace",
        "is_new_burst_start_Paste",
        "is_new_burst_start_Move",
        "burst_time_start",
        "burst_time_end",
        "burst_time_duration",
        "burst_events_Nonproduction",
        "burst_events_Input",
        "burst_events_Remove/Cut",
        "burst_events_Replace",
        "burst_events_Paste",
        "burst_events_Move",
        "word_count_delta_burst",
        "word_count_delta_burst_thin",
        "activity_streak_id",
        "is_new_activity_streak_start",
        "is_new_activity_streak_start_Nonproduction",
        "is_new_activity_streak_start_Input",
        "is_new_activity_streak_start_Remove/Cut",
        "is_new_activity_streak_start_Replace",
        "is_new_activity_streak_start_Paste",
        "is_new_activity_streak_start_Move",
        "is_activity_streak_end",
        "activity_streak_length_thin",

        "down_time",
        "up_time",	
        "action_time",	
        "activity_detailed",
        "activity",	
        "activity_Nonproduction",
        "activity_Input",
        "activity_Remove/Cut",
        "activity_Replace",
        "activity_Paste",
        "activity_Move",
        "down_event",	
        "up_event",	
        "text_change",
        "is_thought_delimiting_punctuation",
        "cursor_position",	
        "word_count",

        "cursor_position_vs_max",
        "cursor_position_cummax",
        "has_cursor_position_moved_right",
        "cursor_position_last_space",

        # "is_latest_space",
        # "is_latest_string_end",
        # "n_alphanum_char_added_to_latest_string",
        # "rolling_length_latest_string",
        # "length_latest_string",

        "word_count_lag1",
        "word_count_delta_event",

        "up_time_lag1",
        "latency_time",
        "preceding_pause_time",
        "preceding_pause_time_start_window",
        "rolling_pause_time",
        "rolling_pause_time_fraction",
        "total_pause_time"
        ]]  

In [ ]:
def concatenate_essay_from_logs(df):
    """
    Concatenate essay text from disparate logged input events.
    Expect df to be *one* author's log.
    Adapted from sources: 
        https://www.kaggle.com/code/hiarsl/feature-engineering-sentence-paragraph-features,
        https://www.kaggle.com/code/kawaiicoderuwu/essay-contructor.
    """

    input_events = df.loc[
        (df.activity != 'Nonproduction'), 
        ['activity_detailed', 'cursor_position', 'text_change']
        ].rename(columns={'activity_detailed': 'activity'})

    essay_text = ""
    for input_event in input_events.values:

        activity = input_event[0]
        cursor_position_after_event = input_event[1]
        text_change_log = input_event[2]

        if activity == 'Replace':

            replace_from_to = text_change_log.split(' => ')
            text_add = replace_from_to[1]
            text_remove = replace_from_to[0]
            cursor_position_start_text_change = (
                cursor_position_after_event - len(text_add)
                )
            cursor_position_after_skip_replace = (
                cursor_position_start_text_change + len(text_remove)
            )

            # essayText start: "the blue cat"
            # replace "blue" with "red"
            # "the redblue cat", skip blue
            essay_text = (
                essay_text[:cursor_position_start_text_change] # "the "
                + text_add # "red"
                # essayText value: "the blue cat" 
                # want remaining " cat", NOT "blue cat"
                + essay_text[cursor_position_after_skip_replace:] 
                )

            continue

        if activity == 'Paste':

            cursor_position_start_text_change = (
                cursor_position_after_event - len(text_change_log)
                )

            # essayText start: "the cat"
            # paste "blue " between
            essay_text = (
                essay_text[:cursor_position_start_text_change] # "the " 
                + text_change_log # "blue "
                # essayText value: "the cat"
                + essay_text[cursor_position_start_text_change:]
            )

            continue

        if activity == 'Remove/Cut':
            # similar process to "Replace" action

            text_remove = text_change_log
            cursor_position_after_skip_remove = (
                cursor_position_after_event + len(text_remove)
            )

            essay_text = (
                essay_text[:cursor_position_after_event] 
                + essay_text[cursor_position_after_skip_remove:]
                )

            continue
        
        if "Move" in activity:

            cursor_intervals_raw_str = (
                activity[10:]
                .replace("[", "")
                .replace("]", "")
                )
            cursor_intervals_separate = cursor_intervals_raw_str.split(' To ')
            cursor_intervals_vectors = [
                x.split(', ') 
                for x in cursor_intervals_separate
                ]
            cursor_interval_from = [
                int(x) for x in cursor_intervals_vectors[0]
                ]
            cursor_interval_to = [
                int(x) for x in cursor_intervals_vectors[1]
                ]

            # "the blue cat ran", move "blue" to
            # "the cat blue ran"
            # note: no change in total text length

            if cursor_interval_from[0] != cursor_interval_to[0]:

                if cursor_interval_from[0] < cursor_interval_to[0]:
                    
                    essay_text = (
                        # all text preceding move-impacted window
                        essay_text[:cursor_interval_from[0]] +
                        # skip where moved block _was_,
                        # proceed to end of move-impacted window
                        essay_text[cursor_interval_from[1]:cursor_interval_to[1]] +
                        # add moved block
                        essay_text[cursor_interval_from[0]:cursor_interval_from[1]] + 
                        # all text proceeding move-impacted window
                        essay_text[cursor_interval_to[1]:]
                    )

                # "the cat ran fast", move "ran" to 
                # "ran the cat fast"
                else:

                    essay_text = (
                        # all text preceding move-impacted window
                        essay_text[:cursor_interval_to[0]] + 
                        # add moved block
                        essay_text[cursor_interval_from[0]:cursor_interval_from[1]] +
                        # skip moved block, still within move-impacted window
                        essay_text[cursor_interval_to[0]:cursor_interval_from[0]] + 
                        # all text proceeding move-impacted window
                        essay_text[cursor_interval_from[1]:]
                    )
      
            continue
        

        cursor_position_start_text_change = (
            cursor_position_after_event - len(text_change_log)
            )
        essay_text = (
            essay_text[:cursor_position_start_text_change] 
            + text_change_log
            + essay_text[cursor_position_start_text_change:]
            )
        
    return pd.DataFrame({'id': df['id'].unique(), 'essay': [essay_text]})

In [ ]:
def enrich_logs(X, is_training_run):

#     PUNCTUATION = X_train.loc[(
#         (X_train['activity'] == 'Input')
#         & (~ X_train['text_change'].isin(['q', ' ']))
#         ), 'text_change'].unique()

    X = scrub_activity(X)

    X = enrich_pauses(X)

    X = enrich_time_bursts(X)

    X = enrich_activity_streaks(X)

    X = enrich_word_count(X)

    X = enrich_cursor_position(X)

    X = enrich_word_length(X)

    X = enrich_punctuation(X)

    X = enrich_time_windows(X)

    print("Proceeding to activity onehot encode.")
    X = transform_activity_onehot(X, is_training_run)
    print("Completed activity onehot encode.")

    X = enrich_burst_type(X)

    print("Proceeding to burst type onehot encode")
    X = transform_burst_type_onehot(X, is_training_run)
    print("Completed burst type onehot encode")

    return subset_features(X)


In [ ]:
event_vars_sum = (
    ['activity_' + x for x in ACTIVITY_CATEGORIES] 
    + ['is_new_burst_start'] 
    + ['is_new_burst_start_' + x for x in ACTIVITY_CATEGORIES]
    # + ["is_thought_delimiting_punctuation"]
    + ["is_new_activity_streak_start_" + x for x in ACTIVITY_CATEGORIES]
    )

conti_vars_sum = (
    ['word_count_delta_event']
    + ["preceding_pause_time"]
    )


def aggregate_no_time_dependence_measures(X):

    events_sum_over_time = (
        X
        .groupby('id')
        [event_vars_sum]
        .agg(sum)
        )

    events_sum_over_time['delete_insert_ratio'] = (
        events_sum_over_time['activity_Remove/Cut'] / 
        events_sum_over_time['activity_Input'] 
        )

    conti_sum_over_time = (
        X
        .groupby('id')
        [conti_vars_sum]
        .agg(sum)
        )

    sums_over_time = pd.merge(
        events_sum_over_time,
        conti_sum_over_time,
        how='left',
        left_index=True,
        right_index=True
    )


    centrals_over_time = (
        X
        .groupby('id')
        .agg(
            latency_time_p50 = ('latency_time', np.median),
            pause_time_p50 = ('preceding_pause_time', np.median),
            has_cursor_position_moved_right_mean = ('has_cursor_position_moved_right', 'mean'),
            word_count_delta_burst_mean = ('word_count_delta_burst_thin', 'mean'),
            word_count_delta_burst_p50 = ('word_count_delta_burst_thin', np.median),
            activity_streak_length_mean = ('activity_streak_length_thin', 'mean'),
            cursor_position_vs_max_avg = ('cursor_position_vs_max', 'mean'),
            # length_latest_string_mean = ('length_latest_string', 'mean'),
            # length_latest_string_stddev = ('length_latest_string', np.std)
            )
        )
    

    extremes_over_time = (
        X
        .groupby('id')
        .agg(
            pause_time_max=('preceding_pause_time', 'max'),
            initial_pause_time_max=('preceding_pause_time_start_window', 'max'),
            # approximation to, next longest pause after first long planning pause
            pause_time_p99=('preceding_pause_time', lambda x: x.quantile(0.99)),
            word_count_delta_burst_max=('word_count_delta_burst_thin', 'max'),
            activity_streak_length_max=('activity_streak_length_thin', 'max'),
            total_time=('up_time', 'max'),
            # length_latest_string_max=('length_latest_string', 'max'),
            latency_time_min=('latency_time', 'min'),
            is_time_beyond_expected_max=('is_time_beyond_expected_max', 'max')
            )
        )

    extremes_over_time['is_initial_pause_max_pause'] = (
        extremes_over_time['pause_time_max'] == 
        extremes_over_time['initial_pause_time_max']
        ).astype(int)
    

    from scipy.stats import lognorm

    pause_distr_summary_subjects = []

    for X_subject in [x for _, x in X.groupby('id')]:

        shape, location, scale = lognorm.fit(X_subject['preceding_pause_time'].dropna())

        pause_distr_summary = pd.DataFrame({
            'pauses_lognorm_shape': [shape], 
            'pauses_lognorm_location': [location],
            'pauses_lognorm_scale': [scale]
            })
        pause_distr_summary.index = [X_subject['id'].iloc[0]]
        
        pause_distr_summary_subjects.append(pause_distr_summary)

    distr_params_over_time = pd.concat(pause_distr_summary_subjects, axis=0)


    # aggregates covering final writing product (as opposed to writing process)
    X_groups = [x for _, x in X.groupby('id')]
    essays_text = pd.concat(
        [concatenate_essay_from_logs(x) for x in X_groups], axis=0
        ).reset_index(drop=True)
    del X_groups
    
    # two consecutive newlines should count as one effective
    essays_text['n_paragraphs'] = essays_text['essay'].str.count("[\n]+")

    essays_text['n_sentences'] = essays_text['essay'].str.count("[\.]+|[?]+|[!]+")
    essays_text['sentences'] = essays_text['essay'].str.split("[\.]+|[?]+|[!]+")
    essays_text['words_by_sentence'] = (
        essays_text['sentences'].apply(lambda x: [a.split(" ") for a in x])
    )
    essays_text['n_words_by_sentence'] = (
        essays_text['words_by_sentence']
        .apply(lambda x: np.array([len(a) for a in x]))
    )
    essays_text['words_per_sentence_avg'] = (
        essays_text['n_words_by_sentence']
        .apply(lambda x: x.mean())
    )
    essays_text['words_per_sentence_stddev'] = (
        essays_text['n_words_by_sentence']
        .apply(lambda x: x.std())
    )

    essays_text['n_thought_delimiting_punctuation'] = (
        essays_text
        ['essay']
        .str
        .count("[\.]+|[?]+|[!]+|[,]+|[-]+|[;]+|[:]+")
        )
    essays_text['words'] = essays_text['essay'].str.split(" +", regex=True)
    essays_text["word_count_reconstructed"] = (
        essays_text
        ["words"]
        .apply(lambda x: len(x))
    )
    essays_text["words_per_thought_delimiting_punctuation"] = (
        essays_text["word_count_reconstructed"] / 
        essays_text['n_thought_delimiting_punctuation']
    )
    essays_text["words_len"] = (
        essays_text["words"].apply(lambda x: np.array([len(a) for a in x]))
    )
    essays_text["word_length_avg"] = (
        essays_text["words_len"]
        .apply(lambda x: x.mean())
    )
    essays_text["word_length_max"] = (
        essays_text["words_len"]
        .apply(lambda x: x.max())
    )
    essays_text["word_length_stddev"] = (
        essays_text["words_len"]
        .apply(lambda x: x.std())
    )

    aggregates_essay_text = essays_text[[
        'id',
        'n_paragraphs', 
        'n_sentences', 
        'n_thought_delimiting_punctuation',
        'words_per_sentence_avg', 
        'words_per_sentence_stddev',
        "word_length_avg",
        "word_length_max",
        "word_length_stddev",
        "words_per_thought_delimiting_punctuation"
        ]]
    aggregates_essay_text = aggregates_essay_text.set_index('id')


    aggregates_over_time = pd.merge(
        sums_over_time, 
        centrals_over_time,
        how='left',
        left_index=True,
        right_index=True
        )

    aggregates_over_time = pd.merge(
        aggregates_over_time, 
        extremes_over_time,
        how='left',
        left_index=True,
        right_index=True
        )

    aggregates_over_time = pd.merge(
        aggregates_over_time, 
        distr_params_over_time,
        how='left',
        left_index=True,
        right_index=True
        )
    
    aggregates_over_time = pd.merge(
        aggregates_over_time, 
        aggregates_essay_text,
        how='left',
        left_index=True,
        right_index=True
        )
    

    for var in event_vars_sum:

        aggregates_over_time[var + '_per_s'] = (
            1000 * (aggregates_over_time[var] / aggregates_over_time['total_time'])
            )

    aggregates_over_time = (
        aggregates_over_time
        .assign(
            keystroke_speed = lambda x: (x.activity_Input + x['activity_Remove/Cut']) / x.total_time,
            # words_per_thought_delimiting_punctuation = lambda x: x.word_count_delta_event / x.is_thought_delimiting_punctuation,
            )
        )
    
    
    return aggregates_over_time

In [ ]:
def aggregate_time_variability_measures(X, aggregates_over_time):

    events_by_window = (
        X
        .groupby(['id', 'window_30s'])
        [event_vars_sum]
        .agg(sum)
        .astype(float)
        .fillna(0)
        .reset_index(drop=False)
        )
    events_by_window['delete_insert_ratio'] = (
        events_by_window['activity_Remove/Cut'] / 
        events_by_window['activity_Input'] 
        ).replace(np.inf, np.nan)


    # by default, every categorical time window ever observed across data
    # tabulates for every writer. instead, per writer, truncate to time windows
    # actually used.
    events_by_window['has_activity'] = (
        events_by_window
        [['activity_' + x for x in ACTIVITY_CATEGORIES]].sum(axis=1) 
        > 0
    )
    events_by_window['idx_window_by_id'] = (
        events_by_window
        .groupby('id')
        .cumcount()
    )
    events_by_window['idx_has_activity'] = np.where(
        events_by_window['has_activity'], 
        events_by_window['idx_window_by_id'],
        np.nan
        )
    events_by_window['idx_activity_max'] = (
        events_by_window
        .groupby(['id'])
        ['idx_has_activity']
        .transform('max')
    )
    events_by_window = (
        events_by_window
        .loc[events_by_window['idx_window_by_id'] <= events_by_window['idx_activity_max']]
        .drop(columns=['has_activity', 'idx_has_activity', 'idx_activity_max'])
    )


    # for variability measure more comparable between writers, de-mean by writer. 
    # Ex: higher-throughput writer incurs higher stddev, because values have higher magnitude
    # join method allows for merge on one index column, of multiple possible
    events_by_window = events_by_window.join(
        aggregates_over_time[[x + '_per_s' for x in event_vars_sum]],
        on='id',
        how='left'
        )
    for var in event_vars_sum:
        events_by_window[var + '_time_norm'] = (
            events_by_window[var] / 
            (events_by_window[var + '_per_s'].replace(0, None) * 30)
            ).fillna(1)
    events_by_window.drop(
        columns=[x + '_per_s' for x in event_vars_sum],
        inplace=True
        )


    events_over_time_ren = aggregates_over_time[event_vars_sum]
    events_over_time_ren.columns = [x + "_total" for x in events_over_time_ren.columns]
    events_by_window = events_by_window.join(
        events_over_time_ren, on='id', how='left'
        )
    for var in event_vars_sum:
        events_by_window[var + '_frac_total'] = (
            events_by_window[var] / 
            (events_by_window[var + '_total'].replace(0, None))
            ).fillna(1)
    events_by_window.drop(
        columns=[x + '_total' for x in event_vars_sum],
        inplace=True
        )


    conti_by_window = (
        X
        .assign()
        .groupby(['id', 'window_30s'])
        [conti_vars_sum]
        .agg(sum)
        .astype(float)
        .fillna(0)
        .reset_index(drop=False)
        )
    conti_by_window['idx_window_by_id'] = (
        conti_by_window
        .groupby('id')
        .cumcount()
    )

    conti_over_time_ren = aggregates_over_time[conti_vars_sum]
    conti_over_time_ren.columns = [x + "_total" for x in conti_over_time_ren.columns]
    conti_by_window = conti_by_window.join(
        conti_over_time_ren, on='id', how='left'
        )
    for var in conti_vars_sum:
        conti_by_window[var + '_frac_total'] = (
            conti_by_window[var] / conti_by_window[var + '_total']
            )
    conti_by_window.drop(
        columns=[x + '_total' for x in conti_vars_sum],
        inplace=True
        )


    centrals_by_window = (
        X
        .groupby(['id', 'window_30s'])
        ['cursor_position_vs_max']
        .agg('mean')
        .astype(float)
        .reset_index(drop=False)
        )
    centrals_by_window['idx_window_by_id'] = (
        centrals_by_window
        .groupby('id')
        .cumcount()
    )


    # events table thru writer's final used time window, _not_ last possible
    aggregates_by_window = pd.merge(
        events_by_window, 
        conti_by_window,
        how='inner'
        )

    aggregates_by_window = pd.merge(
        aggregates_by_window, 
        centrals_by_window,
        how='left'
        )
    

    from scipy.stats import entropy

    entropy_vars = [var for var in aggregates_by_window.columns if 'frac_total' in var]
    entropy_by_window = (
        aggregates_by_window
        .groupby(['id'])
        [entropy_vars]
        .agg(lambda x: entropy(x.value_counts()))
        )
    entropy_by_window.columns = [
        x + '_entropy' 
        for x in entropy_by_window.columns
        ]


    sd_by_window = (
        aggregates_by_window
        .drop(columns=['window_30s', 'idx_window_by_id'])
        .groupby(['id'])
        .agg(np.std)
        )
    sd_by_window.columns = [
        x + "_stddev"
        for x in sd_by_window.columns
        ]


    trend_by_window = (
        aggregates_by_window
        .sort_values(['id', 'idx_window_by_id'])
        .drop(columns=['window_30s'])
        .groupby(['id'])
        .corr()
        )
    # extract correlations strictly with time index
    trend_by_window = trend_by_window.xs('idx_window_by_id', level=1)

    vars_drop = (
        [x for x in trend_by_window.columns if 'time_norm' in x]
        + [x for x in trend_by_window.columns if 'frac_total' in x]
        + ['idx_window_by_id']
        )
    trend_by_window = trend_by_window.drop(columns=vars_drop)

    trend_by_window.columns = [
        x + "_ttrend"
        for x in trend_by_window.columns
        ]

    trend_by_window = trend_by_window.fillna(0)


    vari_by_window = pd.merge(
        entropy_by_window,
        sd_by_window,
        how='left',
        left_index=True,
        right_index=True
        )   

    vari_by_window = pd.merge(
        vari_by_window,
        trend_by_window,
        how='left',
        left_index=True,
        right_index=True
        )     
    
    
    return vari_by_window

In [ ]:
def feature_transform_pipeline(X_logs, is_training_run):

    X_logs_enriched = enrich_logs(X_logs, is_training_run)

    aggregates_over_time = aggregate_no_time_dependence_measures(X_logs_enriched)
    vari_by_window = aggregate_time_variability_measures(
        X_logs_enriched, aggregates_over_time 
        )

    X_transform = pd.merge(
        aggregates_over_time,
        vari_by_window,
        how='left',
        left_index=True,
        right_index=True
        )
    
    return X_transform

In [ ]:
# expect train_logs are too large for single batch processing
X_train_logs = extract(PATH_TRAIN_LOGS)

X_train_logs_groups = [x for _, x in X_train_logs.groupby('id')]
del X_train_logs

X_train_logs_chunk1 = X_train_logs_groups[0:1200]
X_train_logs_chunk2 = X_train_logs_groups[1200:]
del X_train_logs_groups

X_train_logs_chunk1 = pd.concat(X_train_logs_chunk1, axis=0)
X_train_logs_chunk2 = pd.concat(X_train_logs_chunk2, axis=0).reset_index(drop=True)


In [ ]:
X_train_chunk1 = feature_transform_pipeline(X_train_logs_chunk1, True)
del X_train_logs_chunk1

In [ ]:
X_train_chunk2 = feature_transform_pipeline(X_train_logs_chunk2, True)
del X_train_logs_chunk2

In [ ]:
X_train = pd.concat([X_train_chunk1, X_train_chunk2], axis=0)
del X_train_chunk1, X_train_chunk2

In [ ]:
X_train.to_pickle("./data/processed/X_train.pkl")

In [ ]:
# X_train = pd.read_pickle("./data/processed/train_logs_enriched.pkl")

In [56]:
import tensorflow as tf
from tensorflow import keras

X_train_logs = extract(PATH_TRAIN_LOGS)
X_train_logs = scrub_activity(X_train_logs)
X_train_logs = [x for _, x in X_train_logs.groupby('id')]
essays_text = pd.concat(
    [concatenate_essay_from_logs(x) for x in X_train_logs],
    axis=0
)

y = pd.read_csv("./data/external/train_scores.csv")
y.rename(columns={'score': 'y'}, inplace=True)
XY = pd.merge(essays_text, y, how='left')
X, y = XY['essay'].to_numpy(), XY['y'].to_numpy()

# in tf Dataset structure, one element is one X-y pair 
XY_train = tf.data.Dataset.from_tensor_slices((X, y))
X_train = XY_train.map(lambda x, y: x)

In [54]:
# essays_text['words'] = essays_text['essay'].str.split(" +", regex=True)
# essays_text["word_count_reconstructed"] = (
#     essays_text
#     ["words"]
#     .apply(lambda x: len(x))
# )
# essays_text["word_count_reconstructed"].describe()

In [55]:
# for inputs, targets in XY_train:
#     print(inputs.shape)
#     print(inputs.dtype)
#     print(inputs)
#     print(targets)

In [57]:
text_vectorization = tf.keras.layers.TextVectorization(
    # with anonymized text, downscale recommended vocabulary size
    # by magnitude
    max_tokens=2000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='tf_idf'
    )

text_vectorization.adapt(X_train)

tfidf_XY_train = XY_train.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)

In [58]:
text_vectorization.get_vocabulary()

['[UNK]',
 'qqqq',
 'qq',
 'qqq',
 'qqqqq',
 'qqqqqq',
 'qqqqqqq',
 'qqqqqqqq',
 'qqqqqqqqq',
 'q',
 'qqqqqqqqqq',
 'qqqqqqqqqqq',
 'qqqqqqqqqqqq',
 'qqqqqqqqqqqqq',
 'qqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqqqqqq',
 'qqqqq\x97qqqq',
 'qqqqqqqqqqq\x97qqq',
 'qâ\x80\x93',
 'qqqqq\x92q',
 'qqqqqq\x92q',
 'qqqqqqqqqqqqqqqqqqqqqqqqqq',
 'â\x80\x93',
 '\x97qq',
 'qâ\x80\x99qqqqqq',
 'qâ\x80\x99q',
 'qÄqq',
 'qqÄqq',
 'qqqâ\x80\x93',
 'qqq\xa0qqqq',
 'qqqqâ\x80\x99q',
 'qqqq\x97qq',
 'qqqq\x97',
 'qqqqqâ\x80\x93',
 'qqqqqÄqq',
 'qqqqq\x97qqqqqq',
 'qqqqqqÂq',
 'qqqqqq\x97qqq\x97qqqq',
 'qqqqqq\x97qqq',
 'qqqqqq\x96qqqqq',
 'qqqqqqq\xa0qqqq',
 'qqqqqqq\x96qqqqqqq',
 'qqqqqqqqqËqq',
 'qqqqqqqqqq\x97qqqqq',
 'qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq',
 'qqqqqqqqqqqqqqqqqqqqqqqqqq

In [61]:
X_train_logs = extract(PATH_TRAIN_LOGS)

In [63]:
X_train_logs['text_change'].value_counts().to_dict()

{'q': 6128208,
 ' ': 1376829,
 'NoChange': 703851,
 '.': 67797,
 ',': 54797,
 '\n': 27977,
 "'": 13192,
 '"': 4659,
 '-': 4492,
 '?': 3507,
 ';': 2905,
 '=': 2288,
 '/': 1354,
 '\\': 1100,
 ':': 813,
 '(': 733,
 '[': 725,
 ')': 660,
 '!': 639,
 'q => q': 238,
 'qqqq => q': 205,
 'qqq => q': 191,
 '>': 177,
 'qqqqqq => q': 165,
 ']': 157,
 'qq => q': 155,
 'qqqqq => q': 142,
 '. ': 137,
 'qqq  => q': 130,
 'qqqq  => q': 120,
 'qqqqqqq => q': 114,
 '$': 112,
 '*': 108,
 'qqq ': 90,
 'qqqqq  => q': 88,
 'qqq': 83,
 'qqqq ': 81,
 'qqqq': 78,
 '&': 77,
 '%': 77,
 'qqqqqqqq => q': 76,
 'qqqqqqqqq => q': 75,
 'qqqqqq  => q': 74,
 'qq  => q': 70,
 '<': 67,
 'qqqqq': 61,
 'qqqqqqq  => q': 60,
 'qqqqqq': 58,
 'qq': 55,
 'qqqqq ': 51,
 'qq ': 49,
 'qqqqqqqqqq => q': 41,
 'qqqqqq ': 41,
 'qqqqqqqq  => q': 41,
 '\n ': 40,
 '+': 40,
 'qqqqqqqqqqq => q': 38,
 'qqqqqqqqq  => q': 37,
 'qqqqqqq': 37,
 '{': 35,
 '_': 34,
 '#': 31,
 "'q": 30,
 'qqqqqqq ': 30,
 '`': 30,
 'qqqqqqqq': 29,
 '@': 26,
 'qqqqqqq